# Basic experiment

In [ ]:
from LocalVolEnv import LocalVol
import gymnasium as gym # type: ignore
from stable_baselines3 import PPO # type: ignore
from stable_baselines3.common.evaluation import evaluate_policy # type: ignore
from stable_baselines3.common.vec_env import DummyVecEnv # type: ignore
from stable_baselines3.common.env_checker import check_env # type: ignore
from stable_baselines3.common.monitor import Monitor # type: ignore
import numpy as np # type: ignore
import random

In [2]:
N = 10
Dynamics  = 'BS'
star_time = 0
T = 1
dT = 1/252
r = 0
mu = [0.05]
sigma = [0.3]
P = [[1]]

LVol = LocalVol(Dynamics = Dynamics, T = T, dT = dT, mu = mu, sigma = sigma, P = P)
LVol.seed(seed=random.seed(10))

env = gym.wrappers.TimeLimit(LVol, max_episode_steps=T)
env = Monitor(env, allow_early_resets=True)

Nepisodes = 100
rew = []
act = []
tradingtimes = []

for i in range(Nepisodes):
    obs = env.reset()
    cont = True
    i = 0
    act.append([])
    tradingtimes.append([])
    reward_episode = 0
    while cont:
        action = 252#env.action_space.sample()
        obs, reward, terminated, truncated, info = LVol.step(action)
        act[-1].append(action)
        reward_episode += reward
        i += 1
        if any([terminated,truncated]):
            cont = False
            tradingtimes[-1].append(env.unwrapped.tradingtimes)
            rew.append(reward_episode)


print(np.mean(rew),np.std(rew))

0.0005091053934821902 5.894828527969956e-05


In [6]:
steps = 100000

path_folder = f"C:/Users/yoshi/OneDrive/Desktop/Research/Optimal Trading Frequency/Optimal-Trading-Frequency/BS_PPO" # PATH to the BS_PPO_Models folder
path = f"{path_folder}/BS_PPO_{str(steps)}_n_regimes_{str(len(mu))}"
for k in range(len(mu)):
    path += f"mu[{str(k)}]_mu{str(int(mu[k]*100))}"

model = PPO.load(path, env = DummyVecEnv([lambda: env]), print_system_info=True)

== CURRENT SYSTEM INFO ==
- OS: Windows-11-10.0.26100-SP0 10.0.26100
- Python: 3.12.9
- Stable-Baselines3: 2.3.2
- PyTorch: 2.6.0+cpu
- GPU Enabled: False
- Numpy: 2.0.2
- Cloudpickle: 3.1.1
- Gymnasium: 0.29.1
- OpenAI Gym: 0.26.2

== SAVED MODEL SYSTEM INFO ==
- OS: Windows-11-10.0.26100-SP0 10.0.26100
- Python: 3.12.9
- Stable-Baselines3: 2.3.2
- PyTorch: 2.6.0+cpu
- GPU Enabled: False
- Numpy: 2.0.2
- Cloudpickle: 3.1.1
- Gymnasium: 0.29.1
- OpenAI Gym: 0.26.2



In [31]:
Nepisodes = 100
rew = []
act = []
tradingtimes = []

for i in range(Nepisodes):
    obs = env.reset()
    obs = obs[0]
    
    #obs = [[obs[0][i] for i in range(len(obs[0]))]]
    cont = True
    i = 0
    act.append([])
    tradingtimes.append([])
    reward_episode = 0
    while cont:
        action = model.predict([obs], deterministic = True)
        obs, reward, terminated, truncated, info = LVol.step(action[0][0])
        act[-1].append(action)
        reward_episode += reward
        i += 1
        if any([terminated, truncated]):
            cont = False
            tradingtimes[-1].append(env.unwrapped.tradingtimes)
            rew.append(reward_episode)

print(np.mean(rew),np.std(rew))


0.0005033821799006766 6.204861222379161e-05


In [32]:
print(act)

[[(array([250]), None), (array([250]), None)], [(array([250]), None), (array([250]), None)], [(array([250]), None), (array([250]), None)], [(array([250]), None), (array([250]), None)], [(array([250]), None), (array([250]), None)], [(array([250]), None), (array([250]), None)], [(array([250]), None), (array([250]), None)], [(array([250]), None), (array([250]), None)], [(array([250]), None), (array([250]), None)], [(array([250]), None), (array([250]), None)], [(array([250]), None), (array([250]), None)], [(array([250]), None), (array([250]), None)], [(array([250]), None), (array([250]), None)], [(array([250]), None), (array([250]), None)], [(array([250]), None), (array([250]), None)], [(array([250]), None), (array([250]), None)], [(array([250]), None), (array([250]), None)], [(array([250]), None), (array([250]), None)], [(array([250]), None), (array([250]), None)], [(array([250]), None), (array([250]), None)], [(array([250]), None), (array([250]), None)], [(array([250]), None), (array([25